In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
#Filter data between these seasons. 
start_season = 2016 
end_season = 2020

In [3]:
player = pd.read_csv('data/player.csv', encoding='ISO-8859-1')

player_stats = pd.read_csv('data/player_stats.csv')


match = pd.read_csv('data/match.csv')[['match_id', 'match_date', 'season']]
match = match.loc[match.match_date!='0000-00-00'] #remove erroneous data #only 1 row
match['start_year'] = match['season'].str[:4].astype(int) #create start_year column
match['match_date'] = pd.to_datetime(match['match_date']) #convert to date
match = match.drop(columns = 'season') #drop season column

In [4]:
player_stats

,player_id,match_id,is_in_starting_11,substitution_on,substitution_off,yellow_card,red_card,is_home_side
0,p1,4,1,Null,Null,Null,Null,0
1,p1,12,1,Null,Null,Null,Null,1
2,p1,24,1,Null,Null,Null,Null,1
3,p1,41,1,Null,Null,Null,Null,0
4,p1,46,1,Null,83',True,Null,0
...,...,...,...,...,...,...,...,...
356460,p999,7462,0,70',Null,Null,Null,1
356461,p999,7493,0,86',Null,True,Null,1
356462,p999,7549,0,Null,Null,Null,Null,1
356463,p999,7577,0,Null,Null,Null,Null,0


In [5]:
# Remove non-numeric characters from the substitution_on/off columns

player_stats['substitution_on'] = player_stats['substitution_on'].str.replace("'", "")
player_stats['substitution_off'] = player_stats['substitution_off'].str.replace("'", "")

def convert_minute(minute_str):
    if pd.isna(minute_str):
        return np.nan
    elif minute_str == 'Null':
        return np.nan
    elif '+' in minute_str:
        minute_str = minute_str.split('+')[1].strip()
        minute_int = int(minute_str) + 90
    else:
        minute_int = int(minute_str)
    return minute_int


player_stats['substitution_on'] = player_stats['substitution_on'].apply(convert_minute)
player_stats['substitution_off'] = player_stats['substitution_off'].apply(convert_minute)

In [6]:
#Merge player_stats to get player_name
player['p_id2'] = player['player_name'].str.lower().str.replace(' ', '')

player1 = pd.merge(player_stats,player[['p_id2', 'player_name', 'position', 'player_id']], on='player_id', how='left')

#Merge player1 with match table to get date of each game played
player2 = pd.merge(player1, match, on ='match_id', how = 'left')

In [7]:
#Filter between seasons we are analysing
player2 = player2.loc[player2.start_year.between(start_season, end_season)]

In [8]:
def calculate_minutes_played(row):
    if row['is_in_starting_11'] == 1 and pd.isnull(row['substitution_off']):
        return 95 #Average of 5 additional minutes
    elif row['is_in_starting_11'] == 1 and pd.notnull(row['substitution_off']):
        return row['substitution_off']
    elif row['is_in_starting_11'] == 0 and pd.isnull(row['substitution_on']):
        return 0
    elif row['is_in_starting_11'] == 0 and pd.notnull(row['substitution_on']) and pd.isnull(row['substitution_off']):
        return 95 - row['substitution_on']
    elif row['is_in_starting_11'] == 0 and pd.notnull(row['substitution_on']) and pd.notnull(row['substitution_off']):
        return row['substitution_off'] - row['substitution_on']
    
player2['minutes_played'] = player2.apply(calculate_minutes_played, axis=1)

#Create game_played column which is 1 if minutes_played is more than 0
player2['game_played'] = player2['minutes_played'].apply(lambda x: 1 if x > 0 else 0) 

#change negative values to 0
player2['minutes_played'] = player2['minutes_played'].apply(lambda x: 0 if x < 0 else x) 

#Take relevant columns
player3 = player2[['p_id2', 'position' ,'match_id','match_date', 'start_year','minutes_played','game_played']]

In [9]:
player3.head()

,p_id2,position,match_id,match_date,start_year,minutes_played,game_played
509,albertomoreno,Defender,14040,2016-08-14,2016.0,95.0,1
510,albertomoreno,Defender,14051,2017-03-12,2016.0,0.0,0
511,albertomoreno,Defender,14067,2016-08-27,2016.0,0.0,0
512,albertomoreno,Defender,14073,2016-09-10,2016.0,2.0,1
513,albertomoreno,Defender,14080,2016-09-16,2016.0,0.0,0


In [10]:
injuries = pd.read_csv('data/injuries.csv', encoding='ISO-8859-1')

# Remove non-alphabetic characters and convert to lowercase
injuries['p_id2'] = injuries['player_name'].replace('[^a-zA-Z]+', '', regex=True).str.lower()


#remove nulls
injuries = injuries.loc[injuries['season_injured'].notna()]

#add start_year column
injuries['start_year'] = ('20' + injuries['season_injured'].str[:2]).astype(int)


# Define a function to extract integers from the duration column
def extract_duration(row):
    duration_str = row['duration']
    match = re.search(r'\d+', duration_str)
    if match:
        return int(match.group())
    else:
        return 0
    
# Apply the function to create a new column with integer values
injuries['duration'] = injuries.apply(extract_duration, axis=1)

#Remove injuries which are sicknesses 
injuries = injuries.loc[~injuries.injury.isin(['Corona virus', 'Ill', 'Cold','Food Poisoning', 'Quarantine'])]

#Filter only injuries between start and end year

injuries = injuries.loc[injuries.start_year.between(start_season,end_season)]

#take relevant columns
injuries = injuries[['p_id2','start_year', 'injury',
       'injured_since', 'injured_until', 'duration']]

In [11]:
# Grouping player_dataframe by player_id
# group by p_id2 and start_year and aggregate by summing minutes_played and counting number of rows
injuries_grouped = injuries.groupby(['p_id2', 'start_year']).agg({'duration': 'sum'})

# rename the columns and reset index
injuries_grouped = injuries_grouped.rename(columns={'duration': 'season_days_injured'})
injuries_grouped = injuries_grouped.reset_index()

# create total minutes played column
total_days_injured = injuries.groupby('p_id2').agg({'duration': 'sum'}).rename(
    columns={'duration': 'total_days_injured'})

# merge the total minutes and games played columns with the grouped dataframe
injuries_grouped = injuries_grouped.merge(total_days_injured, on='p_id2')
df_injuries = injuries_grouped
df_injuries

,p_id2,start_year,season_days_injured,total_days_injured
0,aaronconnolly,2018,77,161
1,aaronconnolly,2019,13,161
2,aaronconnolly,2020,71,161
3,aaroncresswell,2016,95,226
4,aaroncresswell,2018,87,226
...,...,...,...,...
3105,zlatanibrahimovic,2016,210,579
3106,zlatanibrahimovic,2017,84,579
3107,zlatanibrahimovic,2018,36,579
3108,zlatanibrahimovic,2019,39,579


In [12]:
fifa_16 = pd.read_csv('data/players_16.csv')
fifa_17 = pd.read_csv('data/players_17.csv')
fifa_18 = pd.read_csv('data/players_18.csv')
fifa_19 = pd.read_csv('data/players_19.csv')
fifa_20 = pd.read_csv('data/players_20.csv')
fifa_21 = pd.read_csv('data/players_21.csv')

In [13]:
#Function to extract name from url
def extract_name_url(df):
    # Copy the original DataFrame to avoid modifying it directly
    new_df = df.copy()
    
    # Extract the name from the player_url using a regular expression
    pattern = r'/player/\d+/(.*)/'
    new_df['p_id2'] = df['player_url'].apply(lambda x: re.findall(pattern, x)[0])
    
    # Remove any non-alphabetic characters and convert to lowercase
    new_df['p_id2'] = new_df['p_id2'].apply(lambda x: re.sub('[^a-zA-Z]', '', x.lower()))
    
    return new_df

#Function to apply extract_name_url and take relevant columns 

def clean_fifa_data(df):
    
    #Filter relevant columns only
    relevant_cols = ['sofifa_id', 'player_url', 'dob', 'height_cm', 'weight_kg', 'nationality', 'work_rate',
                    'pace', 'physic', 'overall']
    df = df[relevant_cols]
    df = extract_name_url(df)
    return df

In [14]:
#Apply functions
fifa_16 = clean_fifa_data(fifa_16)
fifa_17 = clean_fifa_data(fifa_17)
fifa_18 = clean_fifa_data(fifa_18)
fifa_19 = clean_fifa_data(fifa_19)
fifa_20 = clean_fifa_data(fifa_20)
fifa_21 = clean_fifa_data(fifa_21)

In [15]:
#Take average for all fifa df's and combine into 1 df

# Concatenate all the dataframes into a single dataframe
fifa_all = pd.concat([fifa_16, fifa_17, fifa_18, fifa_19, fifa_20, fifa_21])

# Group the concatenated dataframe by sofifa_id
grouped = fifa_all.groupby('sofifa_id')

# Compute the mean of float/int columns and the mode of non-numeric columns for each group
aggregated = grouped.agg(lambda x: x.mean() if x.dtype=='float64' or x.dtype=='int64' else x.mode()[0])

# Create a new dataframe called 'fifa' using the aggregated data
fifa = pd.DataFrame(aggregated.to_records())

# Rename columns as needed
fifa = fifa.rename(columns={'index': 'sofifa_id', 'overall': 'fifa_rating'})

#take only relevant columns
fifa = fifa[['p_id2','dob', 'height_cm', 'weight_kg',
       'nationality', 'work_rate', 'pace', 'physic', 'fifa_rating']]

In [16]:
fifa.head()

,p_id2,dob,height_cm,weight_kg,nationality,work_rate,pace,physic,fifa_rating
0,giovannipasquale,1982-01-05,182.000000,72.000000,Italy,Medium/High,71.000000,71.000000,69.000000
1,luisgarciafernandez,1981-02-06,177.666667,66.333333,Spain,Medium/Medium,56.000000,63.333333,70.000000
2,joecole,1981-11-08,176.000000,73.000000,England,High/Low,56.000000,54.000000,72.000000
3,manuelherrerayague,1981-09-29,182.000000,75.000000,Spain,Medium/Medium,NaN,NaN,71.000000
4,andresiniestalujan,1984-05-11,171.000000,68.000000,Spain,High/Medium,68.333333,59.000000,85.666667


In [17]:
print('Unique players in player dataframe:',player3.p_id2.nunique())
print('Unique players in injury dataframe:',df_injuries.p_id2.nunique())
print('Unique players in FIFA dataframe:', fifa.p_id2.nunique())

Unique players in player dataframe: 1411
Unique players in injury dataframe: 1212
Unique players in FIFA dataframe: 37909


In [18]:
# List of players to be included in our analysis
player_list = injuries['p_id2'].tolist() 

#Filter player and fifa dataframes to include relevant player_list
player4 = player3.loc[player3.p_id2.isin(player_list)]
fifa2 = fifa.loc[fifa.p_id2.isin(player_list)]

print('Unique players on all 3 datasets:',pd.merge(
    player4,injuries, on='p_id2' ,how = 'inner').merge(fifa2, on = 'p_id2', how = 'inner').p_id2.nunique())

Unique players on all 3 datasets: 738


In [19]:
# Grouping player_dataframe by player_id
# group by p_id2 and start_year and aggregate by summing minutes_played and counting number of rows
grouped = player4.groupby(['p_id2', 'start_year']).agg({'minutes_played': 'sum', 'game_played': 'sum', 'match_id': 'count'})

# rename the columns
grouped = grouped.rename(columns={
    'minutes_played': 'season_minutes_played', 'game_played': 'season_games_played', 'match_id':'season_matches_in_squad'})

# reset the index
grouped = grouped.reset_index()

# create total minutes played column
total_minutes_played = player4.groupby('p_id2').agg({'minutes_played': 'sum'}).rename(
    columns={'minutes_played': 'total_minutes_played'})

# create total games played column
total_games_played = player4.groupby('p_id2').agg({'game_played': 'sum'}).rename(columns={'game_played': 'total_games_played'})
    
# merge the total minutes and games played columns with the grouped dataframe
grouped = grouped.merge(total_minutes_played, on='p_id2').merge(total_games_played, on='p_id2')

# Group by p_id2 and start_year
grouped2 = player4.groupby(['p_id2', 'start_year'])

df_playtime = grouped

In [20]:
#Merge relevant dataframes
df1 = df_injuries.merge(df_playtime, on = ['p_id2', 'start_year'], how= 'inner')
df2 = df1.merge(fifa, on ='p_id2', how = 'inner')

#Add column for position
df = df2.merge(player[['p_id2', 'position']], on = 'p_id2', how = 'inner')

print('Final number of players to be analyzed:',df.p_id2.nunique())

Final number of players to be analyzed: 604


In [21]:
df.head()

,p_id2,start_year,season_days_injured,total_days_injured,season_minutes_played,season_games_played,season_matches_in_squad,total_minutes_played,total_games_played,dob,height_cm,weight_kg,nationality,work_rate,pace,physic,fifa_rating,position
0,aaronconnolly,2019,13,161,1312.0,24,28,2148.0,41,2000-01-28,175.333333,75.666667,Republic of Ireland,Medium/Low,72.333333,58.0,63.000000,Forward
1,aaronconnolly,2020,71,161,836.0,17,28,2148.0,41,2000-01-28,175.333333,75.666667,Republic of Ireland,Medium/Low,72.333333,58.0,63.000000,Forward
2,aaroncresswell,2016,95,226,2247.0,26,27,13368.0,149,1989-12-15,171.666667,66.000000,England,High/Medium,74.333333,67.0,75.333333,Defender
3,aaroncresswell,2018,87,226,1680.0,20,27,13368.0,149,1989-12-15,171.666667,66.000000,England,High/Medium,74.333333,67.0,75.333333,Defender
4,aaroncresswell,2019,35,226,2870.0,31,31,13368.0,149,1989-12-15,171.666667,66.000000,England,High/Medium,74.333333,67.0,75.333333,Defender


In [22]:
df.to_csv('df_injury_player_data.csv', index = False)